# Boxcars Gem Notebook
Below you will find some sample boxcar gem code fragments.

## Env Setup
Before we get started, install the gems we need here:
```bash
gem install boxcars dotenv
```
and then create / edit .env to have SERPAPI_API_KEY and OPENAI_ACCESS_TOKEN

In [1]:
# setup for code below
require "dotenv/load"
require "boxcars"

true

## Examples
Here are several examples of using the boxcar gems. First we will start with individual Boxcars, and then we will move on to using Trains.

### Search using the Google Serp API (set SERPAPI_API_KEY in .env)

In [2]:
# showcase Google Serp search
s = Boxcars::GoogleSearch.new
s.run "what temperature is it in Phoenix?"

Question: what temperature is it in Phoenix?
Answer: High 62F


"High 62F"

### Calculator that uses ruby for hard math

In [3]:
c = Boxcars::Calculator.new
c.run "what is pi to the third power times epsilon?"

> Entering Calculator#run
what is pi to the third power times epsilon?
RubyREPL: puts(Math::PI ** 3 * Math::E)
Answer: 84.28379846823243

84.28379846823243
< Exiting Calculator#run


"84.28379846823243"

### Active Record Example using SQL code generation

In [4]:
# first stub out an in memory sqlite3 and a simple helpdesk example
require 'active_record'
require './helpdesk_sample'

-- create_table("users", {:force=>:cascade})
   -> 0.0165s
-- create_table("comments", {:force=>:cascade})
   -> 0.0004s
-- create_table("tickets", {:force=>:cascade})
   -> 0.0003s


true

In [5]:
# get the answer
boxcar = Boxcars::SQL.new
boxcar.run "how many open tickets for John?"

> Entering data#run
how many open tickets for John?
SELECT COUNT(*) FROM tickets t INNER JOIN users u ON t.user_id = u.id WHERE u.name = 'John' AND t.status = 0;
Answer: [{"COUNT(*)"=>1}]
< Exiting data#run


"Answer: [{\"COUNT(*)\"=>1}]"

In [6]:
boxcar.run "how many comments do we have on closed tickets?"

> Entering data#run
how many comments do we have on closed tickets?
SELECT count(*) FROM comments c INNER JOIN tickets t ON c.ticket_id=t.id WHERE t.status = 0;
Answer: [{"count(*)"=>4}]
< Exiting data#run


"Answer: [{\"count(*)\"=>4}]"

### Active Record Example using ActiveRecord code generation

In [7]:
helpdesk = Boxcars::ActiveRecord.new(name: 'helpdesk', models: [Ticket, User, Comment])
helpdesk.run "how many comments do we have on open tickets?"

> Entering helpdesk#run
how many comments do we have on open tickets?
Comment.where(ticket: Ticket.where(status: 0)).count
Answer: 4
< Exiting helpdesk#run


"Answer: 4"

#### ActiveRecord is Read Only by default

In [8]:
helpdesk.run "Move all of Sally's open tickets to John"

> Entering helpdesk#run
Move all of Sally's open tickets to John
computing change count with: Ticket.where(user_id: Sally.id, status: 0).count
Ticket.where(user_id: Sally.id, status: 0).update_all(user_id: John.id)
Answer: nil
< Exiting helpdesk#run


"Answer: nil"

#### But you can make changes if you want!

In [11]:
rw_helpdesk = Boxcars::ActiveRecord.new(read_only: false, name: 'Helpdesk', models: [Ticket, User, Comment])
rw_helpdesk.run "Move all of Sally's open tickets to John"

> Entering Helpdesk#run
Move all of Sally's open tickets to John
computing change count with: Ticket.where(status: 0, user_id: User.where(name: "Sally")).count
Pending Changes: 2
Ticket.where(status: 0, user_id: User.where(name: "Sally")).update_all(user_id: User.where(name: "John"))
Answer: 2
< Exiting Helpdesk#run


"Answer: 2"

#### Or you can use a callback for approval if changes are needed

In [13]:
Ticket.update_all(user_id: User.find_by(name: 'John').id)
my_approver = Proc.new do |change_count, code|
    puts ">>> Approving #{change_count} changes <<<"
    true
end
ap_helpdesk = Boxcars::ActiveRecord.new(approval_callback: my_approver, name: 'Helpdesk', models: [Ticket, User, Comment])
ap_helpdesk.run "Move all of John's open tickets to Sally"

> Entering Helpdesk#run
Move all of John's open tickets to Sally
computing change count with: Ticket.where(user_id: User.find_by(name: 'John'), status: 0).count
Pending Changes: 3
>>> Approving 3 changes <<<
Ticket.where(user_id: User.find_by(name: 'John'), status: 0).update_all(user_id: User.find_by(name: 'Sally'))
Answer: 3
< Exiting Helpdesk#run


"Answer: 3"

## Putting it all together - Trains
Trains uses a series of Boxcars to compute an answer by composing answers from the union of queries to individual boxcars as needed.
### a one car train
This is similar to just running Boxcar.run

In [14]:
c = Boxcars::Calculator.new
train = Boxcars.train.new(boxcars: [c])
train.run "what is pi squared?"

> Entering Zero Shot#run
what is pi squared?
> Entering Calculator#run
pi * pi
Answer: 9.869604401089358
< Exiting Calculator#run
Observation: Answer: 9.869604401089358
I now know the final answer
Final Answer: 9.869604401089358
< Exiting Zero Shot#run


"9.869604401089358"

### a two car train
Here we have a HelpDesk ActiveRecord database and a calculator at our disposal to get an answer

In [15]:
c = Boxcars::Calculator.new
helpdesk = Boxcars::ActiveRecord.new(name: 'helpdesk', models: [Ticket, User, Comment])
train = Boxcars.train.new(boxcars: [c, helpdesk])
train.run "the number of open helpdesk tickets that John commented on times 2 pi?"

> Entering Zero Shot#run
the number of open helpdesk tickets that John commented on times 2 pi?
> Entering helpdesk#run
How many tickets did John comment on?
Comment.where(user_id: User.where(name: 'John').first.id).count
Answer: 2
< Exiting helpdesk#run
Observation: Answer: 2
> Entering Calculator#run
2 * 2 pi
RubyREPL: puts(2 * 2 * Math::PI)
Answer: 12.566370614359172

12.566370614359172
< Exiting Calculator#run
Observation: 12.566370614359172
I now know the final answer
Final Answer: 12.566370614359172
< Exiting Zero Shot#run


"12.566370614359172"

#### a three car train

In [16]:
c = Boxcars::Calculator.new
search = Boxcars::GoogleSearch.new
helpdesk = Boxcars::ActiveRecord.new(name: 'helpdesk', models: [Ticket, User, Comment])
train = Boxcars.train.new(boxcars: [c, helpdesk, search])
train.run "the number of open helpdesk tickets that John commented on times 2 pi plus the temp in Phoenix"

> Entering Zero Shot#run
the number of open helpdesk tickets that John commented on times 2 pi plus the temp in Phoenix
> Entering helpdesk#run
Search for John's comments in the helpdesk tickets
Comment.where(user_id: User.find_by(name: 'John').id)
Answer: #<ActiveRecord::Relation [#<Comment id: 1, content: "This is a comment", user_id: 1, ticket_id: 1, created_at: "2023-02-23 23:40:50.536049000 +0000", updated_at: "2023-02-23 23:40:50.536049000 +0000">, #<Comment id: 2, content: "This is johns second comment", user_id: 1, ticket_id: 1, created_at: "2023-02-23 23:40:50.536994000 +0000", updated_at: "2023-02-23 23:40:50.536994000 +0000">]>
< Exiting helpdesk#run
Observation: Answer: #<ActiveRecord::Relation [#<Comment id: 1, content: "This is a comment", user_id: 1, ticket_id: 1, created_at: "2023-02-23 23:40:50.536049000 +0000", updated_at: "2023-02-23 23:40:50.536049000 +0000">, #<Comment id: 2, content: "This is johns second comment", user_id: 1, ticket_id: 1, created_at: "2023-02-23

"68.2831853071796 open helpdesk tickets that John commented on times 2 pi plus the temp in Phoenix"

#### Add as many as you want.